In [83]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(2019)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from keras import metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
import os

In [84]:
train_dir = "/home/dai/Downloads/my project/Brain Tumor Images Dataset-20200103T163459Z-001/Brain Tumor Images Dataset/training_set"
validation_dir = "/home/dai/Downloads/my project/Brain Tumor Images Dataset-20200103T163459Z-001/Brain Tumor Images Dataset/training_set/"

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))


Found 140 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


# model

In [85]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dropout(0.1, seed=2019),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2, seed=2019),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('Organic') 
    # and 1 for the other ('Recycled')
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [86]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 72, 72, 16)        2320      
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 36, 36, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 34, 34, 16)        2320      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 17, 17, 16)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 15, 15, 16)       

In [87]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [88]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=10,
                              epochs=10,
                              validation_steps=50,
                              verbose=1)

Epoch 1/10
10/10 [==============================] - 9s 929ms/step - loss: 0.6956 - accuracy: 0.5500 - val_loss: 0.6619 - val_accuracy: 0.5580
Epoch 2/10
10/10 [==============================] - 9s 918ms/step - loss: 0.6518 - accuracy: 0.6150 - val_loss: 0.5283 - val_accuracy: 0.7460
Epoch 3/10
10/10 [==============================] - 9s 917ms/step - loss: 0.4550 - accuracy: 0.7650 - val_loss: 0.4465 - val_accuracy: 0.7450
Epoch 4/10
10/10 [==============================] - 9s 898ms/step - loss: 0.4635 - accuracy: 0.7650 - val_loss: 0.2888 - val_accuracy: 0.8920
Epoch 5/10
10/10 [==============================] - 9s 917ms/step - loss: 0.4031 - accuracy: 0.8100 - val_loss: 0.2726 - val_accuracy: 0.9150
Epoch 6/10
10/10 [==============================] - 9s 900ms/step - loss: 0.3092 - accuracy: 0.8800 - val_loss: 0.2099 - val_accuracy: 0.9230
Epoch 7/10
10/10 [==============================] - 9s 907ms/step - loss: 0.2363 - accuracy: 0.8900 - val_loss: 0.2475 - val_accuracy: 0.8940
Epoch 

In [89]:
model.save('model_new.h5')

In [90]:
model = load_model('model_new.h5')

# prediction

In [105]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

model = load_model('model_new.h5')
model.compile(loss='binary_crossentropy',

              optimizer='rmsprop',

              metrics=['accuracy'])

img = cv2.imread('/home/dai/Downloads/my project/Brain Tumor Images Dataset-20200103T163459Z-001/Brain Tumor Images Dataset/test_set/hemmorhage_data/028.png')
img = cv2.resize(img,(150,150))
img = np.reshape(img,[1,150,150,3])
img = tf.cast(img, tf.float32)
classes = model.predict_classes(img)
print(classes)
i = 1

for things in classes:  
    if(things == 0):
        print('%d.It is cancer'%(i))
    else:
        print('%d.Not cancer'%(i))
    i = i + 1

[[0]]
1.It is cancer


In [97]:
from tensorflow.keras.models import load_model

import cv2

import numpy as np

model = load_model('model_new.h5')

model.compile(loss='binary_crossentropy',

              optimizer='rmsprop',

              metrics=['accuracy'])

img = cv2.imread('/home/dai/Downloads/my project/Brain Tumor Images Dataset-20200103T163459Z-001/Brain Tumor Images Dataset/test_set/non_hemmorhage_data/128.png')
img = cv2.resize(img,(150,150))
img = np.reshape(img,[1,150,150,3])
img = tf.cast(img, tf.float32)
classes = model.predict_classes(img)
print(classes)
i = 1

for things in classes:  
    if(things == 0):
        print('%d.It is cancer'%(i))
    else:
        print('%d.Not cancer'%(i))
    i = i + 1


[[1]]
1.Not cancer
